In [54]:
import numpy as np
from tqdm import tqdm
from time import time

import torchvision
from torchvision import models, transforms

import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter

In [55]:
def accuracy(yhat,y):
    # si y encode les indexes
    if len(y.shape)==1 or y.size(1)==1:
        return (torch.argmax(yhat,1).view(y.size(0),-1)== y.view(-1,1)).double().mean()
    # si y est encodé en onehot
    return (torch.argmax(yhat,1).view(-1) == torch.argmax(y,1).view(-1)).double().mean()

def train(model,epochs,train_loader,test_loader,feature_extract=False):
    model = model.to(device)
    writer = SummaryWriter(f"{TB_PATH}/{model.name}")
    
    params_to_update = model.parameters()
    print("params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    optim = torch.optim.Adam(params_to_update,lr=1e-3)
    
    print(f"running {model.name}")
    loss = nn.CrossEntropyLoss()
    for epoch in tqdm(range(epochs)):
        cumloss, cumacc, count = 0, 0, 0
        model.train()
        for x,y in train_loader:
            optim.zero_grad()
            x,y = x.to(device), y.to(device)
            yhat = model(x)
            l = loss(yhat,y)
            l.backward()
            optim.step()
            cumloss += l*len(x)
            cumacc += accuracy(yhat,y)*len(x)
            count += len(x)
        writer.add_scalar('loss/train',cumloss/count,epoch)
        writer.add_scalar('accuracy/train',cumacc/count,epoch)
        if epoch % 1 == 0:
            model.eval()
            with torch.no_grad():
                cumloss, cumacc, count = 0, 0, 0
                for x,y in test_loader:
                    x,y = x.to(device), y.to(device)
                    yhat = model(x)
                    cumloss += loss(yhat,y)*len(x)
                    cumacc += accuracy(yhat,y)*len(x)
                    count += len(x)
                writer.add_scalar(f'loss/test',cumloss/count,epoch)
                writer.add_scalar('accuracy/test',cumacc/count,epoch)

def set_parameter_requires_grad(model, feature_extract):
    if feature_extract:
        for name,p in model.named_parameters():
            p.requires_grad = False    

def get_test_data(dataloader, size):
    X_test, Y_test = next(iter(dataloader))
    batch_size = len(X_test)
    n = size//batch_size
    for i, batch in enumerate(dataloader):
        if i < n:
            X_tmp, Y_tmp = batch
            X_test = torch.cat((X_test, X_tmp), 0)
            Y_test = torch.cat((Y_test, Y_tmp), 0)
    return X_test, Y_test

In [56]:
TB_PATH = "/tmp/logs/sceance2"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# yolov5 = models.yolov518(pretrained=True)
yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# yolov5.fc = nn.Linear(512, 10)

print(yolov5.eval())

set_parameter_requires_grad(yolov5, True)

In [57]:
input_size = 224
batch_size = 128

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

transformyolov5Train=transforms.Compose([ # Cette fois on utilise pas de grayscale car nous avons un gros modele pré-entrainé
        transforms.RandomResizedCrop(input_size), # selection aléatoire d'une zone de la taille voulue (augmentation des données en apprentissage)
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
transformyolov5Test=transforms.Compose([
        transforms.Resize(input_size), # selection de la zone centrale de la taille voulue
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

yolov5_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transformyolov5Train)
yolov5_trainloader = torch.utils.data.DataLoader(yolov5_trainset, batch_size=batch_size, pin_memory=True, shuffle=True)

yolov5_testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transformyolov5Test)
yolov5_testloader = torch.utils.data.DataLoader(yolov5_testset, batch_size=batch_size, pin_memory=True, shuffle=True)

In [58]:
## Entraînement du réseau
yolov5.name = "yolov5"
# train(yolov5, 1, yolov5_trainloader, yolov5_testloader)

In [59]:
## Accuracy
X_test, Y_test = get_test_data(yolov5_testloader, 1000) 
X_test, Y_test = X_test.to(device), Y_test.to(device)
# print("Acc for yolov5 transfer learning :", accuracy(yolov5(X_test), Y_test))

In [60]:
from matplotlib import pyplot as plt

images = yolov5(X_test[:3])
images.shape
# grid_img = torchvision.utils.make_grid(images, nrow=11).cpu()
# print(grid_img.shape)
# plt.figure(figsize=(24, 12))
# plt.imshow(grid_img.permute(1, 2, 0))
# plt.axis('off')

In [61]:
def classFilter(classdata):
    classes = []  # create a list
    for i in range(classdata.shape[0]):         # loop through all predictions
        classes.append(classdata[i].argmax())   # get the best classification location
    return classes  # return classes (int)

def YOLOdetect(output_data):  # input = interpreter, output is boxes(xyxy), classes, scores
    output_data = output_data[0]                # x(1, 25200, 7) to x(25200, 7)
    boxes = np.squeeze(output_data[..., :4])    # boxes  [25200, 4]
    scores = np.squeeze( output_data[..., 4:5]) # confidences  [25200, 1]
    classes = classFilter(output_data[..., 5:]) # get classes
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    x, y, w, h = boxes[..., 0], boxes[..., 1], boxes[..., 2], boxes[..., 3] #xywh
    xyxy = [x - w / 2, y - h / 2, x + w / 2, y + h / 2]  # xywh to xyxy   [4, 25200]

    return xyxy, classes, scores  # output is boxes(x,y,x,y), classes(int), scores(float) [predictions length]

In [62]:
"""Output data"""
output_data = images[0].cpu().detach().numpy()
xyxy, classes, scores = YOLOdetect(output_data) #boxes(x,y,x,y), classes(int), scores(float) [25200]
# print(xyxy)

In [63]:
for t in (20,40,60,80,100,120,400,1000):
    t0 = time()
    yolov5(X_test.cpu()[:t])
    print("FPS:", t, " --> seconds:", (time() - t0))

In [64]:
import os
PATH = "./"
torch.save(yolov5.state_dict(), os.path.join(PATH,"yolov5s.pth"))
model=yolov5

In [65]:
test_size = 100
dummy_input = torch.randn(test_size, 3, input_size, input_size)  
torch.onnx.export(model,   
                  dummy_input, 
                  str(PATH+"yolov5s.onnx"),
                  export_params=True,
                  do_constant_folding=True, 
                  opset_version=11,
                  input_names = ['modelInput'],
                  output_names = ['modelOutput'])

In [66]:
# !pip install onnx
# !pip install onnxruntime

import onnx
import onnxruntime
from onnx import numpy_helper
import numpy as np

X_test = X_test[:test_size].cpu()

t0 = time()
pred = model(X_test).cpu()
print("Time for ",test_size," images without ONNX inference", (time() - t0))
print(np.array(pred).shape)

sess = onnxruntime.InferenceSession(str(PATH+"yolov5s.onnx"))
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name
t0 = time()
pred = sess.run([output_name], {input_name: np.array(X_test).astype(np.float32)})[0]
print("Time for ",test_size," images with ONNX inference", (time() - t0))
print(np.array(pred).shape)